# Imports

In [215]:
import numpy as np
import os
import math
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import csv
import json
from scipy.stats import wasserstein_distance
import statistics
from tqdm import tqdm
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm

# Data Loading Functions

In [100]:
def read_stats_data(name, mechanics, folder):
    data = {}
    for mech in mechanics:
        data[mech] = []
    data["endTurn"] = []
    file_names = [f for f in os.listdir(folder) if name in f]
    for fn in file_names:
        with open(os.path.join(folder,fn)) as f:
            json_data = json.load(f)
            if "results" in json_data:
                json_data = json_data["results"]
            for playtrace in json_data:
                for mech in mechanics:
                    if mech in playtrace["frequencies"]:
                        data[mech].append(playtrace["frequencies"][mech])
                    else:
                        data[mech].append(0.0)
                data["endTurn"].append(playtrace["levelReport"]["turnsTaken"])
    return pd.DataFrame(data)

def build_users_data(folder, mechanics, users):
    full_data = read_stats_data(users[0], mechanics, folder)
    full_data['id'] = users[0]
    for user in users[1:]:
        user_data = read_stats_data(user, mechanics, folder)
        user_data['id'] = user
        full_data = pd.concat([full_data, user_data])
    full_data.set_index('id', inplace=True)
    return full_data

def read_questionaire(name, folder):
    return

# Helper Functions

In [210]:
def get_mechanics(folders):
    uniques = set()
    for folder in folders:
        filenames = [f for f in os.listdir(folder) if ".json" in f]
        for fn in filenames:
            with open(os.path.join(folder,fn)) as f:
                json_data = json.load(f)
                if "results" in json_data:
                    json_data = json_data["results"]
                for playtrace in json_data:
                    for key in playtrace["frequencies"].keys():
                        uniques.add(key)
    return list(uniques)

def get_users(folders):
    uniques = set()
    for folder in folders:
        filenames = [f for f in os.listdir(folder) if ".json" in f]
        for fn in filenames:
            uuid = fn.split(".")[0]
            uniques.add(uuid)
    return list(uniques)

def get_majority_voting(clf, data):
    labels = clf.predict(data).reshape((-1,3))
    result = []
    for row in labels:
        vals,counts= np.unique(row, return_counts=True)
        index = np.argmax(counts)
        result.append(vals[index])
    return np.array(result)

# Building Data

In [182]:
mechanics = get_mechanics(["results", "valid_study"])
users = get_users(["valid_study"])

mk_data = read_stats_data("MK", mechanics, "results")
tc_data = read_stats_data("TC", mechanics, "results")
r_data = read_stats_data("R", mechanics, "results")
user_data = build_users_data("valid_study", mechanics, users)

# Building Data Set

In [183]:
input_data = pd.concat([mk_data, tc_data, r_data], ignore_index=True).to_numpy()
output_data = pd.Series([0]*len(mk_data) + [1]*len(tc_data) + [2]*len(r_data)).to_numpy()
avg_test_set = user_data.groupby('id').mean().to_numpy()
maj_test_set = user_data.to_numpy()

# Training Classifier

In [230]:
dt = tree.DecisionTreeClassifier(max_depth=5)
dt = dt.fit(input_data, output_data)
print(f"Decision Tree Training Score: {dt.score(input_data, output_data)}")

knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(input_data, output_data)
print(f"K Nearest Neighbor Training Score: {knn.score(input_data, output_data)}")

svc = svm.SVC(gamma='auto', decision_function_shape='ovo')
svc = svc.fit(input_data, output_data)
print(f"Support Vector Machine Training Score: {svc.score(input_data, output_data)}")

Decision Tree Training Score: 0.825
K Nearest Neighbor Training Score: 0.9066666666666666
Support Vector Machine Training Score: 0.8996666666666666


# Predicting Users

In [231]:
dt_avg_labels = dt.predict(avg_test_set)
knn_avg_labels = knn.predict(avg_test_set)
svc_avg_labels = svc.predict(avg_test_set)
print(f"Average DT vs KNN: {(dt_avg_labels == knn_avg_labels).sum()/len(avg_test_set)}")
print(f"Average DT vs SVC: {(dt_avg_labels == svc_avg_labels).sum()/len(avg_test_set)}")
print(f"Average KNN vs SVC: {(knn_avg_labels == svc_avg_labels).sum()/len(avg_test_set)}")
dt_maj_labels = get_majority_voting(dt, maj_test_set)
knn_maj_labels = get_majority_voting(knn, maj_test_set)
svc_maj_labels = get_majority_voting(svc, maj_test_set)
print(f"Majority DT vs KNN: {(dt_maj_labels == knn_maj_labels).sum()/len(avg_test_set)}")
print(f"Majority DT vs SVC: {(dt_maj_labels == svc_maj_labels).sum()/len(avg_test_set)}")
print(f"Majority KNN vs SVC: {(knn_maj_labels == svc_maj_labels).sum()/len(avg_test_set)}")

Average DT vs KNN: 0.638095238095238
Average DT vs SVC: 0.6714285714285714
Average KNN vs SVC: 0.8714285714285714
Majority DT vs KNN: 0.6238095238095238
Majority DT vs SVC: 0.6619047619047619
Majority KNN vs SVC: 0.819047619047619
